In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from lib.vllm import start_vllm_server

model = "NousResearch/Hermes-2-Theta-Llama-3-8B"

shutdown_server, client = await start_vllm_server(
    disable_log_requests=True,
    model=model,
)

/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.54it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.28it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.68it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.55it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.53it/s]



In [11]:
shutdown_server()

False

In [1]:
import glob
import subprocess

model_dir = subprocess.run(
    "HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download NousResearch/Hermes-2-Theta-Llama-3-8B",
    shell=True,
    capture_output=True,
    text=True,
).stdout.strip()

print(model_dir)

/home/ubuntu/.cache/huggingface/hub/models--NousResearch--Hermes-2-Theta-Llama-3-8B/snapshots/57a73110702e7b05ba3f39fef36297454c680725


In [2]:
import os
from torchtune.training.checkpointing import FullModelHFCheckpointer

output_dir = "./models/test"
os.makedirs(output_dir, exist_ok=True)

checkpointer = FullModelHFCheckpointer(
    checkpoint_dir=model_dir,
    checkpoint_files=glob.glob(f"{model_dir}/*.safetensors")
    + glob.glob(f"{model_dir}/*.pt"),
    output_dir=output_dir,
    model_type="LLAMA3",  # type: ignore
)
state_dict = checkpointer.load_checkpoint()
state_dict.keys()

dict_keys(['model'])

In [3]:
import torch
import bitsandbytes
from dataclasses import dataclass
import bitsandbytes.optim
from torchtune.modules.loss import CEWithChunkedOutputLoss
from typing import Iterable
from torchtune.models.llama3_1 import llama3_1_8b


@dataclass
class Batch:
    tokens: torch.Tensor  # (batch_size, seq_len)
    mask: torch.Tensor  # (batch_size, seq_len, seq_len)


def batches(batch_size: int, sequence_length: int) -> Iterable[Batch]:
    while True:
        yield Batch(
            tokens=torch.randint(
                0,
                model.tok_embeddings.weight.shape[0],
                size=(batch_size, sequence_length),
            ).to("cuda"),
            # causal mask
            mask=torch.tril(torch.ones(sequence_length, sequence_length))
            .unsqueeze(0)
            .to(torch.bfloat16)  # Convert mask to bf16
            .to("cuda"),
        )


torch.set_float32_matmul_precision("high")
model = llama3_1_8b()
model.load_state_dict(state_dict["model"])
model.to(torch.bfloat16).to("cuda")
model.compile()
optimizer = bitsandbytes.optim.PagedAdamW8bit(model.parameters(), lr=2e-5)
loss_fn = CEWithChunkedOutputLoss()
loss_fn.compute_cross_entropy = torch.compile(loss_fn.compute_cross_entropy)


for i, batch in enumerate(batches(1, 4096)):
    logits = model.forward(batch.tokens, mask=batch.mask)
    assert isinstance(logits, torch.Tensor)
    # shift tokens right to create targets
    targets = batch.tokens[:, 1:]
    preds = logits[:, :-1, :]

    loss = loss_fn(
        preds.chunk(loss_fn.num_output_chunks, dim=1),
        targets,
    )
    print(f"Loss: {loss.item():.4f}")
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i > 3:
        break

W1031 22:57:29.403000 124506728208192 torch/fx/experimental/symbolic_shapes.py:4449] [0/0] xindex is not in var_ranges, defaulting to unknown range.


Loss: 12.0643


W1031 22:57:54.877000 124491669702208 torch/fx/experimental/symbolic_shapes.py:4449] [0/0] xindex is not in var_ranges, defaulting to unknown range.


Loss: 12.1835
Loss: 11.1322
Loss: 6.4802
Loss: 3.6116


In [4]:
for i, batch in enumerate(batches(1, 4096)):
    logits = model.forward(batch.tokens, mask=batch.mask)
    assert isinstance(logits, torch.Tensor)
    # shift tokens right to create targets
    targets = batch.tokens[:, 1:]
    preds = logits[:, :-1, :]

    loss = loss_fn(
        preds.chunk(loss_fn.num_output_chunks, dim=1),
        targets,
    )
    print(f"Loss: {loss.item():.4f}")
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i > 3:
        break

Loss: 2.2667
Loss: 1.8741
Loss: 1.4779
Loss: 1.1809
Loss: 1.0327
